<a href="https://colab.research.google.com/github/FR-Schwartz/IDS705_Team10/blob/main/source/explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use this Noteboook to convert images in the nii.gz format to the Tensorflow protobuf format.

In [ ]:
from google.colab import drive
import os
import shutil
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import nibabel as nib
drive.mount('/content/MyDrive/')
os.chdir('/content/MyDrive/MyDrive/IDS705_Final') #change to file path on your disk

Drive already mounted at /content/MyDrive/; to attempt to forcibly remount, call drive.mount("/content/MyDrive/", force_remount=True).


In [ ]:
#Change subfolder id to checkout different images
subfolders = os.listdir("Data/Train")
print("Number of subfolders:", len(subfolders))

Number of subfolders: 1251


In [ ]:
#Helper Functions to convert nii files to tfrecords
def float_feature(value):
  return tf.train.Feature(float_list = tf.train.FloatList(value = value))

def int64_feature(value):
  return tf.train.Feature(int64_list = tf.train.Int64List(value = value))

def load_one_sample(image_path):
  t1 = nib.load(image_path + "_t1.nii.gz").get_fdata()
  t2 = nib.load(image_path + "_t2.nii.gz").get_fdata()
  t1ce = nib.load(image_path + "_t1ce.nii.gz").get_fdata()
  flair = nib.load(image_path + "_flair.nii.gz").get_fdata()
  image = np.stack([t1, t2, t1ce, flair], 3)
  label = nib.load(image_path + "_seg.nii.gz").get_fdata().astype(int)
  return image, label

def create_example(image_path):
  image, label = load_one_sample(image_path)
  image, label = image.ravel(), label.ravel()
  feature = {'image': float_feature(image),
             'label': int64_feature(label)}
  example = tf.train.Example(features = tf.train.Features(feature = feature))
  return example

#Use this to load data from record to runtime in the form of x=image , y=segmentation_class
def parse_tfrecord(example):
  feature = {'image': tf.io.FixedLenFeature([240, 240, 155, 4], tf.float32),
             'label': tf.io.FixedLenFeature([240, 240, 155], tf.int64)}
  parsed_example = tf.io.parse_single_example(example, feature)
  return parsed_example

In [ ]:
This cell runs conversions to tfrecord one by one. Takes like 4 hrs
for sf in subfolders:
  train_writer = tf.io.TFRecordWriter(f'/content/MyDrive/MyDrive/IDS705_Final/Data/train_tf/{sf}.tfrecord')
  example = create_example(f'Data/Train/{sf}/{sf}')
  train_writer.write(example.SerializeToString())
  train_writer.close()